In [1]:
import pandas as pd

In [2]:
file_path = "Resources/purchase_data.csv"
purchase_data = pd.read_csv(file_path)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


Part 1: Review data per column to determine if clean up is needed and display the count of total number of players

In [3]:
#Identify any incomplete rows
purchase_data.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
#Review datatypes 
purchase_data.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [5]:
#Use pd.to_nuermic() method to convert the datatype of the Price column
purchase_data['Price'] = pd.to_numeric(purchase_data['Price'])

In [6]:
#Verify that datatype change to Price column worked
purchase_data['Price'].dtype

dtype('float64')

In [7]:
#Display any unique values and count for screen name column
purchase_data.loc[:, 'SN'].value_counts()

Lisosia93       5
Iral74          4
Idastidru52     4
Lisim78         3
Tyidaim51       3
               ..
Iarallo65       1
Lisirra25       1
Sondadarya58    1
Iskosia90       1
Mindossa76      1
Name: SN, Length: 576, dtype: int64

In [8]:
counts = len(purchase_data["SN"].drop_duplicates())
counts_df = pd.DataFrame({'Total Players':[counts]})
counts_df

,Total Players
0,576


Part 2 - Purchasing Analysis (Total): Run basic calculations to obtain the number of unique items, averge purchase price, total number of purchases, and total revenue. 
    Create a summary data frame to hold the results and display the summary data frame with clean formatting.

In [9]:
#Calculate the number of unique Items in the DataFrame
un_items = len(purchase_data["Item ID"].unique())

In [10]:
#Calculate the average purchase price
avg_price = purchase_data["Price"].mean()

In [11]:
#Calculate the total number of purchases
purchase_total = len(purchase_data["Purchase ID"].unique())

In [12]:
#Calculate the total revenue
total_sum = purchase_data["Price"].sum()

In [13]:
#Place all of the data found into a summary DataFrame
data = {
    'Number of Unique Items': [un_items],
    'Average Price': [avg_price],
    'Number of Purchases': [purchase_total],
    'Total Revenue': [total_sum]
}

summary1 = pd.DataFrame(data)
summary1
format_dict = {'Average Price':'${0:,.2f}', 'Total Revenue': '${:,.2f}'}
summary1.style.format(format_dict)

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


Part 3 - Gender Demographics: Determine the percentage and count of male players, female player, and of Other/Non-Disclosed

In [14]:
#Remove any duplicate screen names as before
count_m = purchase_data.drop_duplicates(subset = "SN", keep="first")

In [15]:
#Determine total count of males
total_m = len(count_m.loc[purchase_data["Gender"] == "Male"])

In [16]:
#Determine percentage of male players
total_m_p = (total_m / counts)*100

In [17]:
#Determine total count of females
total_f = len(count_m.loc[purchase_data["Gender"] == "Female"])

In [18]:
#Determine percentage of female players
total_f_p = (total_f / counts)*100

In [19]:
#Determine total count of Other/Non-Disclosed
total_o = len(count_m.loc[purchase_data["Gender"] == "Other / Non-Disclosed"])

In [20]:
#Determine percentage of Other/Non-Disclosed players
total_o_p = (total_o / counts)*100

In [21]:
#Place all of the gender demographics into a summary DataFrame

gendem = pd.DataFrame(columns=['Total Count', 'Percentage of Players'],
                                  index = ['Male', 'Female', 'Other/Non-disclosed'])
gendem.loc['Male'] = pd.Series({
    'Total Count': total_m,
    'Percentage of Players': total_m_p})
gendem.loc['Female'] = pd.Series({
    'Total Count': total_f,
    'Percentage of Players': total_f_p,})
gendem.loc['Other/Non-disclosed'] = pd.Series({
    'Total Count': total_o,
    'Percentage of Players': total_o_p,})

format2 = {'Total Count': '{:,.0f}', 'Percentage of Players': '{:,.2f}%'}
gendem.style.format(format2)

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other/Non-disclosed,11,1.91%


Part 4 - Purchasing Analysis (Gender): Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person, etc. by gender. Create a clean and formatted summary data frame to hold the results. MUST USE GROUPBY METHOD (GENDER).

In [22]:
#For purchase analysis, must use groupby method
grouped_df = purchase_data.groupby(['Gender'])

In [23]:
#Determine data calculations with groupby method
count = grouped_df["SN"].count()

In [24]:
price = grouped_df["Price"].mean()

In [25]:
total = grouped_df["Price"].sum()

In [26]:
#Find average price per person
clean_df = purchase_data.drop_duplicates(subset='SN', keep='first')
group2 = clean_df.groupby(['Gender'])

In [27]:
avg_ppp = (grouped_df["Price"].sum() / group2["Age"].count())

In [28]:
#Display the summary in a data frame
#.map() method used to manipulate pandas Series

summary_df4 = pd.DataFrame({
    'Purchase Count': count,
    'Average Purchase Price': price,
    'Total Purchase Value': total,
    'Avg Total Purchase per Person': avg_ppp})

#Apply formatting
summary_df4['Average Purchase Price'] = summary_df4['Average Purchase Price'].map('${:.3}'.format)
summary_df4['Total Purchase Value'] = summary_df4['Total Purchase Value'].map('${:,.2f}'.format)
summary_df4['Avg Total Purchase per Person'] = summary_df4['Avg Total Purchase per Person'].map('${:.3}'.format)

summary_df4.head()

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.2,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


Part 5 - Age Demographics: Establish BINS for ages. Categorize the existing players using the age bins (i.e. use pd.cut() method). Calculate the numbers and percentages by age group and create a summary data frame to hold the reults.

In [29]:
# purchase_data.drop_duplicates(subset='SN', keep='first')

In [30]:
#Create the bins in which age data will be held
bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200]
# bins = [0, 10, 15, 20, 25, 30, 35, 40, 200]
group_names = ["less than 10", "10-14",  "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
# purchase_data.loc[:, 'Bins']=pd.cut(purchase_data.loc[:, 'Age'], bins=bins)
# purchase_data

In [31]:
purchase_data["Age Count"] = pd.cut(purchase_data["Age"], bins, labels=group_names, include_lowest=True)

In [32]:
group_age = purchase_data.groupby("Age Count")

In [33]:
age_total_count = group_age["SN"].nunique()
age_total_count

Age Count
less than 10     17
10-14            22
15-19           107
20-24           258
25-29            77
30-34            52
35-39            31
40+              12
Name: SN, dtype: int64

In [41]:
#Calculate total numbers and percentages by age group
age_ppp2 = age_total_count / counts #may need to rerun total counts of players

In [42]:
#Create a summary data frame to hold the results
sum_dict = {
    "Total Count": age_total_count,
    "Percentage of Players": age_ppp2}

agedem_df = pd.DataFrame(sum_dict)
agedem_df

,Total Count,Percentage of Players
Age Count,,
less than 10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


In [ ]:
# clean_data2 = purchase_data.drop_duplicates(subset='SN', keep='first')

Part 6: 

In [ ]:
#Create the bins in which age data will be held
bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 200]
# bins = [0, 10, 15, 20, 25, 30, 35, 40, 200]
group_names = ["<10", "10-14",  "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
# purchase_data.loc[:, 'Bins']=pd.cut(purchase_data.loc[:, 'Age'], bins=bins)
# purchase_data

In [ ]:
#Slice the data any place it into bins
pd.cut(purchase_data["Age"], bins, labels=group_names).head()

In [ ]:
# Place the data series into a new column inside of the DataFrame
purchase_data["Age"] = pd.cut(purchase_data["Age"], bins, labels=group_names)
purchase_data.head()

In [ ]:
# Create a GroupBy object based upon "View Group"
age_demo = purchase_data.groupby("Age")

In [ ]:
# Purchase count by age analysis
print(age_demo["Age"].count())

In [ ]:
# Get the average of each column within the GroupBy object
age_demo[["Age", "Price"]].mean()

In [ ]:
# Get the total purchase value of each column within the GroupBy object
age_demo[["Age", "Price"]].sum()

In [ ]:
# Get the average total purchase value per person of each column within the GroupBy object
# age_demo[["Age", "Price"]].sum()
avg_ppp2 = age_demo